In [7]:
import pandas as pd
import numpy as np
from numpy import arctan as atan, sqrt, sin, cos

### Augmented measures

The original sensor data $a_f$, $a_l$, $a_v$ (front, vertical, and lateral sensors, respectively) are augmented messures as described in Wickramasinghe et al (2017, appendix B). Firstly angles $\theta$, $\alpha$, and $\beta$ are defined as follows:


$\theta \approx \tan^{-1}(a_f / \sqrt{a_v^2 + a_l ^2})$

$\alpha \approx \tan^{-1}(a_l / a_v)$

$\beta \approx \tan^{-1}(a_l / a_f)$


Then approximate acceleration signals $a_x$, $a_y$, and $a_z$ are derived as shown below:

$a_x \approx a_v\sin(\theta) + a_f\cos(\theta)$

$a_y \approx a_v\sin(\alpha) + a_l\cos(\alpha)$

$a_z \approx 1 + a_v\cos(\theta)\cos(\alpha) + a_l\sin(\alpha) + a_f\sin(\theta)$

### Features

Features are taken over a window $X_{[t_i - \delta t, t_i]}$:

| Feature | Description |
|:-:|:-|
| $a_f$ | Frontal acceleration |
| $a_l$ | Lateral acceleration |
| $a_v$ | Vertical acceleration |
| $\theta$ | Angle on Sagittal plane |
| $\alpha$ | Angle on Coronal plane |
| $\beta$ | Angle on Transverse plane |
| $a_x$ | Anteroposterior acceleration |
| $a_y$ | Mediolateral acceleration |
| $a_z$ | Dorsvental acceleration |
| $\mbox{mean}$$(a_f)$; $\mbox{mean}$$(a_l)$; $\mbox{mean}$$(a_v)$ |
| $\mbox{max}$$(a_f)$; $\mbox{max}$$(a_l)$; $\mbox{max}$$(a_v)$ |
| $\mbox{min}$$(a_f)$; $\mbox{min}$$(a_l)$; $\mbox{min}$$(a_v)$ |
| $\mbox{corr}$$(a_f\mbox{,}a_l)$; $\mbox{corr}$$(a_f\mbox{,}a_v)$; $\mbox{corr}$$(a_l\mbox{,}a_v)$ |
| $\mbox{mean}$ $(a_x)$; $\mbox{mean}$ $(a_y)$; $\mbox{mean}$ $(a_z)$ | 
| $\mbox{mean}$$(\theta)$; $\mbox{mean}$$(\alpha)$; $\mbox{mean}$$(\beta)$ |
| $\mbox{max}$$(\theta)$; $\mbox{max}$$(\alpha)$; $\mbox{max}$$(\beta)$ |
| $\mbox{min}$$(\theta)$; $\mbox{min}$$(\alpha)$; $\mbox{min}$$(\beta)$ |

In [40]:
from pathlib import Path

import pandas as pd

DATA_ROOT = Path('..') / 'data'

dfs = []
activity_labels = ['bed', 'chair', 'lying', 'ambulating']
default_names = ['time', 'front', 'vertical', 'lateral', 'sensor_id', 'rssi', 'phase', 'frequency', 'activity']
for data_file in Path(DATA_ROOT).rglob('d[12]p??[FM]'):
    df = pd.read_csv(data_file, names=default_names)
    df['activity_label'] = df['activity'].apply(lambda i: activity_labels[i - 1])
    df['gender_label'] = str(data_file)[-1]
    df['participant'] = data_file.name
    
    # Add a column indicating order of the activities for a particiapnt.
    df = df.sort_values(by=['time'])
    df['activity_sequence'] = (df['activity'].shift(1) != df['activity']).cumsum()
    dfs.append(df)

sensor_df = pd.concat(dfs, axis='index')
sensor_df = sensor_df.sort_values(by=['participant', 'time'])

sensor_df.head()

,time,front,vertical,lateral,sensor_id,rssi,phase,frequency,activity,activity_label,gender_label,participant,activity_sequence
0,0.0,0.27203,1.00820,-0.082102,1,-63.5,2.4252,924.25,1,bed,M,d1p01M,1
0,0.0,0.27203,1.00820,-0.082102,1,-63.5,2.4252,924.25,1,bed,M,d1p01M,1
1,0.5,0.27203,1.00820,-0.082102,1,-63.0,4.7369,921.75,1,bed,M,d1p01M,1
1,0.5,0.27203,1.00820,-0.082102,1,-63.0,4.7369,921.75,1,bed,M,d1p01M,1
2,1.5,0.44791,0.91636,-0.013684,1,-63.5,3.0311,923.75,1,bed,M,d1p01M,1


In [37]:
def get_frame(df, t, delta_t):
    frame_df = df[(df['time'] >= t) & (df['time'] < (t + delta_t))]
    
    if not len(frame_df):
        return None

    t = frame_df['time']
    a_f = frame_df['front']
    a_l = frame_df['lateral']
    a_v = frame_df['vertical']

    theta = atan(a_f / sqrt(a_v**2 + a_l**2))
    alpha = atan(a_l / a_v)
    beta = atan(a_l / a_f)

    a_x = a_v * sin(theta) + a_f * cos(theta)
    a_y = a_v * sin(alpha) + a_l * cos(alpha)
    a_z = 1 + a_v * cos(theta) * cos(alpha) + a_l * sin(alpha) + a_f * sin(theta)
    
    
    frame = dict({
        't': t[0],
        'a_f': a_f[0],
        'a_l': a_l[0],
        'a_v': a_v[0],
        'theta': theta[0],
        'alpha': alpha[0],
        'beta': beta[0],
        'a_x': a_x[0],
        'a_y': a_y[0],
        'a_z': a_z[0],
        'mean_a_f': a_f.mean(),
        'mean_a_l': a_l.mean(),
        'mean_a_v': a_v.mean(),
        'max_a_f': a_f.max(),
        'max_a_l': a_l.max(),
        'max_a_v': a_v.max(),
        'min_a_f': a_f.min(),
        'min_a_l': a_l.min(),
        'min_a_v': a_v.min(),
    })
    return pd.DataFrame(frame)


In [38]:
participants = sensor_df['participant'].unique()
participant = participants[0]

participant_df = sensor_df[sensor_df['participant'] == participant]



In [39]:
get_frame(participant_df, 0, 5)

,t,a_f,a_l,a_v,theta,alpha,beta,a_x,a_y,a_z,mean_a_f,mean_a_l,mean_a_v,max_a_f,max_a_l,max_a_v,min_a_f,min_a_l,min_a_v
0,0.0,0.27203,-0.082102,1.0082,0.262712,-0.081255,-0.293119,0.524526,-0.163662,2.047706,0.347407,-0.054409,0.967206,0.44791,-0.013684,1.0082,0.27203,-0.082102,0.91636
0,0.0,0.27203,-0.082102,1.0082,0.262712,-0.081255,-0.293119,0.524526,-0.163662,2.047706,0.347407,-0.054409,0.967206,0.44791,-0.013684,1.0082,0.27203,-0.082102,0.91636
